In [1]:
import json
import pygraphviz as pgv

## Context

In [2]:
f_context = open('mock_data/test_prop_grap_data_001.json')
context = json.load(f_context)

In [3]:
context_list=context["context"].replace(" ","").split(";")[0:-1]

## Node Dictionary

In [4]:
f_output = open('mock_data/test_prop_graph_001.json')
data = json.load(f_output)

In [5]:
data.keys()

dict_keys(['context', 'facts', 'graph', 'dilemmas'])

In [6]:
nodes=list(data["graph"].keys())
nodes

['c', 'e', 'f', '-x', 'y', 'z']

In [7]:
nodes_reverse=nodes[::-1]
nodes_reverse

['z', 'y', '-x', 'f', 'e', 'c']

In [8]:
data["graph"]["z"][0]["body"][0]["name"]

'y'

In [9]:
node_rules=dict()
rules=dict()

In [10]:
for node in nodes_reverse:
    
    length=len(data["graph"][node])
    
    rules_list=[]
    for rule_id in range(length):
        rule_name=data["graph"][node][rule_id]["name"]
        rules_list.append(rule_name)

        sub_nodes=[]
        for item in data["graph"][node][rule_id]["body"]:
            if item["sign"]==True:
                sub_node=item["name"]
            else:
                sub_node="-"+item["name"]
            sub_nodes.append(sub_node)
        rules[rule_name]=sub_nodes
    
    node_rules[node]=rules_list

In [11]:
node_rules

{'z': ['R8'],
 'y': ['R6', 'R7'],
 '-x': ['R5'],
 'f': ['R3'],
 'e': ['R2'],
 'c': ['R1']}

In [12]:
rules

{'R8': ['y', 'f', 'g'],
 'R6': ['a', 'd', 'e', '-x'],
 'R7': ['e', 'g'],
 'R5': ['a', 'c'],
 'R3': ['a', 'd', 'e'],
 'R2': ['c', 'd'],
 'R1': ['a', 'b']}

## Graph Builder

In [13]:
graph=[]
global graph

def graph_builder(start_point,node_rules, rules):
    temp_rules=node_rules[start_point]
    for temp_rule in temp_rules:
        edge=temp_rule+","+start_point
        graph.append(edge)
    for temp_rule in temp_rules:    
        nodes=rules[temp_rule]
        for node in nodes:
            edge=node+","+temp_rule
            graph.append(edge)
            
    for temp_rule in temp_rules:    
        nodes=rules[temp_rule]
        for node in nodes: 
            if node not in context_list:
                graph_builder(node,node_rules, rules)

In [14]:
start_point=list(node_rules.keys())[0]

In [15]:
graph_builder(start_point,node_rules, rules)

In [16]:
G = pgv.AGraph()

In [17]:
for edge in graph:
    start_node_=edge.split(",")[0]
    end_node_=edge.split(",")[1]
    G.add_edge(start_node_,end_node_)

In [18]:
G.layout()
G.draw("file.png")